In [1]:
import pandas as pd

In [ ]:
# Paths
election_results_path = '../dataFiles/processedData/countypres_2000-2016_processed.csv'
religion_path = '../dataFiles/processedData/religion_by_county_2010_processed.csv'
demographics_path = '../dataFiles/processedData/FinalDemographics_cleaned.xlsx'
hispanic_data_path = '../dataFiles/processedData/2010HispanicOnly.xlsx'
education_path = '../dataFiles/processedData/education_processed.xlsx'
income_path = '../dataFiles/processedData/income_final.xlsx'

# Get Data
election_results = pd.read_csv(election_results_path)
religion = pd.read_csv(religion_path)
demographics = pd.read_excel(demographics_path, sheet_name='Sheet1')
hispanic_data = pd.read_excel(hispanic_data_path, sheet_name='Sheet1')
education = pd.read_excel(education_path, sheet_name='Sheet1')
incomes = pd.read_excel(income_path, sheet_name='Sheet1')

In [ ]:
# Make Election Results FIPS 5 Digits
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
election_results['FIPS'] = election_results['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
election_results.insert(len(election_results.columns), 'Add Zero FIPS', '0' + election_results['FIPS'])
election_results.loc[election_results['FIPS'].str.len()==4, 'FIPS'] = election_results.loc[election_results['FIPS'].str.len()==4, 'Add Zero FIPS']
del election_results['Add Zero FIPS']

election_results['totalvotes'] = election_results['totalvotes']
election_results.insert(len(election_results.columns), 'dem_votes', election_results['totalvotes']*election_results['dem_vote_share'])
election_results.insert(len(election_results.columns), 'rep_votes', election_results['totalvotes']*election_results['rep_vote_share'])
election_results.insert(len(election_results.columns), 'other_votes', election_results['totalvotes']*election_results['other_vote_share'])
# Change %s To Doubles
election_results['dem_vote_share'] = election_results['dem_vote_share']*100
election_results['rep_vote_share'] = election_results['rep_vote_share']*100
election_results['other_vote_share'] = election_results['other_vote_share']*100
election_results['republican_minus_democratic'] = election_results['republican_minus_democratic']*100
election_results.head()

In [ ]:
# Merge Religion
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
religion['FIPS'] = religion['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
religion.insert(len(religion.columns), 'Add Zero FIPS', '0' + religion['FIPS'])
religion.loc[religion['FIPS'].str.len()==4, 'FIPS'] = religion.loc[religion['FIPS'].str.len()==4, 'Add Zero FIPS']
del religion['Add Zero FIPS']
# Make Religion Same For 2008, 2012, 2016
religion_2008 = religion.copy()
religion_2012 = religion.copy()
religion_2016 = religion.copy()
religion_2008['Year'] = '2008'
religion_2012['Year'] = '2012'
religion_2016['Year'] = '2016'
religion = religion_2008.append(religion_2012).append(religion_2016).reset_index(drop=True)
# Make FIPS/Year Lookup Key
religion.insert(len(religion.columns), 'FIPS/Year',religion['Year'].astype(str) + '/' + religion['FIPS'])
# Merge Religion Using Lookup Key
religion.rename(columns={'Other':'Other Religion'}, inplace=True)
election_results = election_results.merge(religion[['FIPS/Year','Evangelical Protestant','Black Protestant', 'Mainline Protestant','Catholic','Orthodox','Other Religion']], how='left', on='FIPS/Year')
# Change %s to Doubles
election_results['Evangelical Protestant'] = election_results['Evangelical Protestant']*100
election_results['Black Protestant'] = election_results['Black Protestant']*100
election_results['Mainline Protestant'] = election_results['Mainline Protestant']*100
election_results['Catholic'] = election_results['Catholic']*100
election_results['Orthodox'] = election_results['Orthodox']*100
election_results['Other Religion'] = election_results['Other Religion']*100
election_results.head()

In [ ]:
# Merge Demographics
del demographics['FIPS/Year']
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
demographics['FIPS'] = demographics['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
demographics.insert(len(religion.columns), 'Add Zero FIPS', '0' + demographics['FIPS'])
demographics.loc[demographics['FIPS'].str.len()==4, 'FIPS'] = demographics.loc[demographics['FIPS'].str.len()==4, 'Add Zero FIPS']
del demographics['Add Zero FIPS']
# Make FIPS/Year Lookup Key
demographics.insert(len(demographics.columns), 'FIPS/Year',demographics['year'].astype(str) + '/' + demographics['FIPS'])
demographics.insert(len(demographics.columns), '% Single Parent Households',demographics['% Households: Family Households: Other Family: Male Householder, No Wife Present'] + demographics['% Households: Family Households: Other Family: Female Householder, No Husband Present'])

#Get Columns
columns_to_get = ['FIPS/Year',
'Total Population',
'Population Density (Per Sq. Mile)',
'% Total Population: Male', 
'% Total Population: Female',
'% Total Population: Under 18 Years',
'% Total Population: 18 to 34 Years',
'% Total Population: 35 to 64 Years',
'% Total Population: 65 and Over',
'% Total Population: White Alone',
'% Total Population: Black or African American Alone',
'% Total Population: American Indian and Alaska Native Alone',
'% Total Population: Asian Alone',
'% Total Population: Native Hawaiian and Other Pacific Islander Alone',
'% Total Population: Some Other Race Alone',
'% Total Population: Two or More Races',
'% Population 15 Years and Over: Never Married',
'% Population 15 Years and Over: Now Married (Not Including Separated)',
'% Population 15 Years and Over: Separated',
'% Population 15 Years and Over: Widowed',
'% Population 15 Years and Over: Divorced',
'Average Household Size',
'Gini Index',
'% Civilian Population 18 Years and Over: Veteran',
'% Civilian Population 18 Years and Over: Nonveteran',
'% Single Parent Households']

# Merge Demographics Using Lookup Key
election_results = election_results.merge(demographics[columns_to_get], how='left', on='FIPS/Year')
election_results.head()

In [ ]:
# Demographics- Get Hispanic
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
hispanic_data['FIPS'] = hispanic_data['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
hispanic_data.insert(len(hispanic_data.columns), 'Add Zero FIPS', '0' + hispanic_data['FIPS'])
hispanic_data.loc[hispanic_data['FIPS'].str.len()==4, 'FIPS'] = hispanic_data.loc[hispanic_data['FIPS'].str.len()==4, 'Add Zero FIPS']
del hispanic_data['Add Zero FIPS']
hispanic_data_2008 = hispanic_data[['FIPS', '% Total population: Hispanic or Latino']].copy()
hispanic_data_2012 = hispanic_data[['FIPS', '% Total population: Hispanic or Latino']].copy()
hispanic_data_2016 = hispanic_data[['FIPS', '% Total population: Hispanic or Latino']].copy()
hispanic_data_2008.insert(len(hispanic_data_2008.columns), 'Year', 2008)
hispanic_data_2012.insert(len(hispanic_data_2012.columns), 'Year', 2012)
hispanic_data_2016.insert(len(hispanic_data_2016.columns), 'Year', 2016)

hispanic_data = hispanic_data_2008.append(hispanic_data_2012).append(hispanic_data_2016).reset_index(drop=True)

# Make FIPS/Year Lookup Key
hispanic_data.insert(len(hispanic_data.columns), 'FIPS/Year',hispanic_data['Year'].astype(str) + '/' + hispanic_data['FIPS'])
# Merge Education Using Lookup Key
election_results = election_results.merge(hispanic_data[['FIPS/Year','% Total population: Hispanic or Latino']], how='left', on='FIPS/Year')
election_results.head()

In [ ]:
# Clean Education
# Drop Duplicates
education = education.drop_duplicates(subset=['FIPS','Year']).reset_index(drop=True)
# Only Keep 2012 and 2016
education_2012 = education.loc[education['Year']==2012,:].copy().reset_index(drop=True)
education_2016 = education.loc[education['Year']==2016,:].copy().reset_index(drop=True)
# Make 2008 the Same As 2012
education_2008 = education_2012.copy()
education_2008['Year'] = 2008
education = education_2008.append(education_2012).append(education_2016)
# Convert To Percentages
education["High school graduate (includes equivalency)"] = education["High school graduate (includes equivalency)"]/education["Total"]*100
education["Some college or associate's degree"] = education["Some college or associate's degree"]/education["Total"]*100
education["Bachelor's degree"] = education["Bachelor's degree"]/education["Total"]*100
education["Graduate or professional degree"] = education["Graduate or professional degree"]/education["Total"]*100
education.insert(len(education.columns), "College or Above", education["Bachelor's degree"] + education["Graduate or professional degree"])
# Simplify Names
education.rename(columns={"High school graduate (includes equivalency)":'High School'}, inplace=True)
education.rename(columns={"Some college or associate's degree":'Associates or Some College'}, inplace=True)
education.rename(columns={"Bachelor's degree":'College'}, inplace=True)
education.rename(columns={"Graduate or professional degree":'Graduate School'}, inplace=True)

education = education[['FIPS', 'Year', 'High School', 'Associates or Some College', 'College', 'Graduate School', 'College or Above']].copy()

In [ ]:
# Merge Education
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
education['FIPS'] = education['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
education.insert(len(education.columns), 'Add Zero FIPS', '0' + education['FIPS'])
education.loc[education['FIPS'].str.len()==4, 'FIPS'] = education.loc[education['FIPS'].str.len()==4, 'Add Zero FIPS']
del education['Add Zero FIPS']
# Make FIPS/Year Lookup Key
education.insert(len(education.columns), 'FIPS/Year',education['Year'].astype(str) + '/' + education['FIPS'])
# Merge Education Using Lookup Key
election_results = election_results.merge(education[['FIPS/Year','High School', 'Associates or Some College', 'College', 'Graduate School', 'College or Above']], how='left', on='FIPS/Year')
election_results.head()

In [ ]:
# Economic Data
# Make Sure FIPS are 5 Digits
# Make Int FIPS a String
incomes['FIPS'] = incomes['FIPS'].astype(str)
# If FIPS String is 4 Digits Long- Add 0 to Beginning
incomes.insert(len(incomes.columns), 'Add Zero FIPS', '0' + incomes['FIPS'])
incomes.loc[incomes['FIPS'].str.len()==4, 'FIPS'] = incomes.loc[incomes['FIPS'].str.len()==4, 'Add Zero FIPS']
del incomes['Add Zero FIPS']

In [ ]:
# Make 1 DF
incomes_2008 = incomes[['FIPS','Median household income 2010']].copy()
incomes_2012 = incomes[['FIPS','Median Household income 2012']].copy()
incomes_2016 = incomes[['FIPS','Median Household income 2016']].copy()
incomes_2008.rename(columns={'Median household income 2010':'Median Income'}, inplace=True)
incomes_2012.rename(columns={'Median Household income 2012':'Median Income'}, inplace=True)
incomes_2016.rename(columns={'Median Household income 2016':'Median Income'}, inplace=True)
incomes_2008.insert(len(incomes_2008.columns), 'Year', 2008)
incomes_2012.insert(len(incomes_2012.columns), 'Year', 2012)
incomes_2016.insert(len(incomes_2016.columns), 'Year', 2016)
incomes = incomes_2008.append(incomes_2012).append(incomes_2016).reset_index(drop=True)

# Make FIPS/Year Lookup Key
incomes.insert(len(incomes.columns), 'FIPS/Year',incomes['Year'].astype(str) + '/' + incomes['FIPS'])
                 
# Merge Income Using Lookup Key
election_results = election_results.merge(incomes[['FIPS/Year','Median Income']], how='left', on='FIPS/Year')
election_results.head()

In [ ]:
# Export Final Dataframe
output_path  = '../dataFiles/mergedData/election_results_with_variables.xlsx'
election_results.to_excel(output_path,index=False)

In [ ]:
election_results.info()